# Prologue

In [24]:
from pymatgen.core import Structure
from pymatgen.analysis.structure_matcher import StructureMatcher, ElementComparator
from trixs.spectra.core import XAS_Spectrum, XAS_Collation
from trixs.spectra.util import NumpyEncoder
from tqdm import tqdm, tqdm_notebook
from pprint import pprint
import json
import os
import numpy as np
import sklearn
import pandas as pd
import numpy as np
rseed = 42
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold

storage_directory = '/Users/steventorrisi/Documents/TRIXS/data/MP_OQMD_combined'

target_elements_groups=[('Co','O')]#,('Fe','O'),('V','O'),('Cu','O'),
                        #('Ni','O'),('Cr','O'),('Mn','O'),('Ti','O')]
from trixs.machine_learning.benchmarks import precision_recall_matrix, confusion_matrix


In [25]:
data_by_pair = {pair:[] for pair in target_elements_groups}
for pair in target_elements_groups:
    target_file = storage_directory + '/{}_{}_polynomials_pre-post_XY.json'.format(pair[0],pair[1])
    with open(target_file, 'r') as f:
        for line in f.readlines():
            cur_data = json.loads(line)
            if cur_data.get('one_hot_coord'):
                data_by_pair[pair].append(cur_data)
sorted_keys = sorted(list(data_by_pair[('Co','O')][0]['labeled_coefficients'].keys()))

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

for pair in [('Co','O')]:
    X = []
    Y = []    
    for point in data_by_pair[pair]:
        #print(point['labeled_coefficients'])
        #assert False
        X.append([point['labeled_coefficients'][key] for key in sorted_keys])
        Y.append(point['coordination'])

    X = np.array(X)
    Y = np.array(Y)
    
    x_train, x_test, y_train, y_test = \
      train_test_split(X, Y, test_size=0.1,
                   random_state=42)
    x_train, x_valid, y_train, y_valid = \
      train_test_split(x_train, y_train, test_size=0.1,
                       random_state=42)
    
    forest = RandomForestClassifier(random_state=42)
    pprint(forest.get_params())
    
    #model.fit(X,Y)
    ##print(X)
    #print(Y)
    n_estimators = [200,500]
    max_depth=[25,30,35]
    min_samples_split= [2]
    min_samples_leaf = [1]
    max_features = [20,25,30]


    hyperF = dict(n_estimators=n_estimators, max_depth=max_depth,
                 min_samples_split = min_samples_split, 
                 min_samples_leaf = min_samples_leaf,
                 max_features = max_features)
    gridF = GridSearchCV(forest,hyperF, cv = 3, verbose = 2)
    bestF = gridF.fit(x_train,y_train)
    #randF =  RandomizedSearchCV(estimator = forest,
    #                           param_distributions = hyperF, 
    #                           n_iter = 100, cv = 3, 
    #                           verbose=2, random_state=42, 
    #                           n_jobs = -1)
    #bestF = randF.fit(x_train,y_train)


{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}
Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   3.4s
[CV] max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s


[CV]  max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   3.2s
[CV] max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   3.2s
[CV] max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=500 
[CV]  max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   7.0s
[CV] max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=500 
[CV]  max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   7.3s
[CV] max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=500 
[CV]  max_depth=25, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   8.9s
[CV] max_depth=25, max_features=25, min_samples_leaf=1, min_samples_sp

[CV]  max_depth=35, max_features=20, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   6.9s
[CV] max_depth=35, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  max_depth=35, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   3.6s
[CV] max_depth=35, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  max_depth=35, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   3.7s
[CV] max_depth=35, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV]  max_depth=35, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=200, total=   4.0s
[CV] max_depth=35, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=500 
[CV]  max_depth=35, max_features=25, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   8.8s
[CV] max_depth=35, max_features=25, min_samples_leaf=1, min_samples_sp

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  6.2min finished


In [38]:
the_best =bestF.best_estimator_
print(bestF.best_estimator_)

print(the_best.score(x_valid,y_valid))
fits = the_best.predict(x_valid)
#print(the_best.score(x_test,y_test))
print(precision_recall_matrix(fits,y_valid,[4,5,6]))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=25, max_features=20, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
0.8717948717948718
[[0.84       0.89361702 0.86597938]
 [0.77777778 0.64615385 0.70588235]
 [0.9112426  0.95652174 0.93333333]]
